In [1]:
import sys


no_of_cpu = 8
max_cores = 8
executor_mem = '56g'

Job_Name = 'Nishant : ID'

from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = ''
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13059,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:

prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 


In [ ]:
def load_gpdb_jdbc(col_str,dbtable,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates()
   
    return data

In [ ]:
def get_latest_progress():
    
    try: 
        gscPythonOptions = {
                 "url":prod_url,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')=='id')\
            .filter(col('source')=='id')
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2018, 10, 31, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

In [ ]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        


        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
    

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

In [ ]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# This temp table has similar schema as original table but no json field and would not contain duplicates
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark

def update(conn_to,table_name,source,prod_gpdb_spark_options,schema="customermart"):
    conn_to.rollback()
    
    
    table_dict = {
    "id": #table_name
              {
                 "primary_key" : ['source_system_name','source_system_customer_id','activity_id'],
                 "out_columns" : ['source_system_name', 'source_system_customer_id','activity_id','activity_date','id_type', 
                                   'id_value','issuer','date_issued','date_reported','valid_until_date','last_updated_date','verification_flag', 'verification_source', 'active_flag'],
                  "id" : #source/subsegment
                      { 
                          "update_fields" : ['last_updated_date','verification_flag', 'verification_source','active_flag','date_issued','date_reported','valid_until_date']
                      }
              }
    }
    
    
    update_fields = table_dict.get(table_name).get(source).get("update_fields")
    primary_key = table_dict.get(table_name).get("primary_key")
    
    # loading data from temp table to get column names
    df_1 = sqlContext.read.format("greenplum").options(**prod_gpdb_spark_options).option("dbschema",schema)\
    .option("dbtable","{}_staging".format(table_name)).load()
    out_columns = df_1.columns
    out_columns = [col for col in out_columns if "json" not in col ]
    #out_columns = table_dict.get(table_name).get("out_columns")
#     print(out_columns)

    cur_to = conn_to.cursor()  

    # this query insert records from temp table to target table when all the records in temp table are new and not present in target
    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} 
    from {schema}.{table_name}_staging """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    
    # this query will be executed to update the records that are already present in target, new values will be assigned from temp table
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    
    # this query will be executed after update query to insert the remaining records
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
#         update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
#         update_progress(table_name,source,time_filter,records)
    conn_to.commit()


In [ ]:
latest_progress=get_latest_progress()
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-2 #currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

In [ ]:
# master_bulk_df=load_gpdb_jdbc(col_str,"mastercraft_bulk_detail_prod","dc_action_date",time_filter)


col_str=['source_id','source_customer_id','dc_unified_id','dc_action_date','pan','pan_is_verified',
        'cleansed_voterid','voter_id_is_verified', 'driving_license','driving_license_is_verified','passport','voter_id',
        'cleansed_passport','passport_is_verified']

master_bulk_df=load_gpdb_jdbc(col_str,"mastercraft_bulk_detail_prod","dc_action_date",time_filter)
id=master_bulk_df.filter((col('dc_unified_id').isNotNull()) & (col('source_id').isNotNull()))

In [ ]:
id=id.withColumn('source_system_name', col('source_id'))\
.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))\
.withColumn('activity_id', hash(col('pan')))\
.withColumn('activity_date', col('dc_action_date'))\
.withColumn('id_type', lit('pan'))\
.withColumn('id_value', col('pan'))\
.withColumn('issuer', lit(None))\
.withColumn('date_issued', lit(None))\
.withColumn('date_reported', col('dc_action_date'))\
.withColumn('valid_until_date', lit(None))\
.withColumn('last_updated_date', col('dc_action_date'))\
.withColumn('verification_flag', when(col('pan_is_verified').isNull(), lit('UV')).otherwise(lit('IV')))\
.withColumn('verification_source', when(col('pan_is_verified').isNull(), lit(None)).otherwise(lit('Mastercraft')))\
.withColumn('active_flag', lit(None))\
.withColumn('id_json', lit(None))\
.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

In [ ]:
id1=id.select(col('source_system_name'),col('source_system_customer_id'),col('activity_id'),\
                         col('activity_date'),col('id_type'),col('id_value'),\
                         col('issuer'),col('date_issued'),col('date_reported'),col('valid_until_date'),col('last_updated_date'),col('verification_flag'),col('verification_source'),\
                         col('active_flag'),col('id_json'), col('primary_key'))
id1 = id1.withColumn('id_value',when(col('id_value').rlike("[A-Z]{5}[0-9]{4}[A-Z]{1}"),col('id_value'))
                                 .otherwise(lit(None)))

id1=id1.filter(col("id_value").isNotNull())

In [ ]:
id=id.withColumn('source_system_name', col('source_id'))\
.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))\
.withColumn('activity_id', hash(coalesce('cleansed_voterid','voter_id')))\
.withColumn('activity_date', col('dc_action_date'))\
.withColumn('id_type', lit('voterid'))\
.withColumn('id_value', coalesce('cleansed_voterid','voter_id'))\
.withColumn('issuer', lit(None))\
.withColumn('date_issued', lit(None))\
.withColumn('date_reported', col('dc_action_date'))\
.withColumn('valid_until_date', lit(None))\
.withColumn('last_updated_date', col('dc_action_date'))\
.withColumn('verification_flag', when(col('voter_id_is_verified').isNull(), lit('UV')).otherwise(lit('IV')))\
.withColumn('verification_source', when(col('voter_id_is_verified').isNull(), lit(None)).otherwise(lit('Mastercraft')))\
.withColumn('active_flag', lit(None))\
.withColumn('id_json', lit(None))\
.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

In [ ]:
id2=id.select(col('source_system_name'),col('source_system_customer_id'),col('activity_id'),\
                         col('activity_date'),col('id_type'),col('id_value'),\
                         col('issuer'),col('date_issued'),col('date_reported'),col('valid_until_date'),col('last_updated_date'),col('verification_flag'),col('verification_source'),\
                         col('active_flag'),col('id_json'), col('primary_key'))

id2 = id2.withColumn('id_value',when(col('id_value').rlike("^[A-Z]{3}\d{7}$"),col('id_value'))
                                 .otherwise(lit(None)))

id2=id2.filter(col("id_value").isNotNull())

In [ ]:
id_f1= id2.union(id1)

In [ ]:
id=id.withColumn('source_system_name', col('source_id'))\
.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))\
.withColumn('activity_id', hash(col('driving_license')))\
.withColumn('activity_date', col('dc_action_date'))\
.withColumn('id_type', lit('driving license'))\
.withColumn('id_value', col('driving_license'))\
.withColumn('issuer', lit(None))\
.withColumn('date_issued', lit(None))\
.withColumn('date_reported', col('dc_action_date'))\
.withColumn('valid_until_date', lit(None))\
.withColumn('last_updated_date', col('dc_action_date'))\
.withColumn('verification_flag', when(col('driving_license_is_verified').isNull(), lit('UV')).otherwise(lit('IV')))\
.withColumn('verification_source', when(col('driving_license_is_verified').isNull(), lit(None)).otherwise(lit('Mastercraft')))\
.withColumn('active_flag', lit(None))\
.withColumn('id_json', lit(None))\
.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

In [ ]:
id3=id.select(col('source_system_name'),col('source_system_customer_id'),col('activity_id'),\
                         col('activity_date'),col('id_type'),col('id_value'),\
                         col('issuer'),col('date_issued'),col('date_reported'),col('valid_until_date'),col('last_updated_date'),col('verification_flag'),col('verification_source'),\
                         col('active_flag'),col('id_json'), col('primary_key'))

id3 = id3.withColumn('id_value',when(col('id_value').rlike("^(([A-Z]{2}[0-9]{2})( )|([A-Z]{2}-[0-9]{2}))((19|20)[0-9][0-9])[0-9]{7}$"),col('id_value'))
                                 .otherwise(lit(None)))

id3=id3.filter(col("id_value").isNotNull())

In [ ]:
id_f2= id_f1.union(id3)

In [ ]:
id=id.withColumn('source_system_name', col('source_id'))\
.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))\
.withColumn('activity_id', hash(coalesce('cleansed_passport','passport')))\
.withColumn('activity_date', col('dc_action_date'))\
.withColumn('id_type', lit('passport'))\
.withColumn('id_value', coalesce('cleansed_passport','passport'))\
.withColumn('issuer', lit(None))\
.withColumn('date_issued', lit(None))\
.withColumn('date_reported', col('dc_action_date'))\
.withColumn('valid_until_date', lit(None))\
.withColumn('last_updated_date', col('dc_action_date'))\
.withColumn('verification_flag', when(col('passport_is_verified').isNull(), lit('UV')).otherwise(lit('IV')))\
.withColumn('verification_source', when(col('passport_is_verified').isNull(), lit(None)).otherwise(lit('Mastercraft')))\
.withColumn('active_flag', lit(None))\
.withColumn('id_json', lit(None))\
.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

In [ ]:
id4=id.select(col('source_system_name'),col('source_system_customer_id'),col('activity_id'),\
                         col('activity_date'),col('id_type'),col('id_value'),\
                         col('issuer'),col('date_issued'),col('date_reported'),col('valid_until_date'),col('last_updated_date'),col('verification_flag'),col('verification_source'),\
                         col('active_flag'),col('id_json'), col('primary_key'))

id4 = id4.withColumn('id_value',when(col('id_value').rlike("^[A-Za-z][1-9]\\d\\s?\\d{4}[1-9]$"),col('id_value'))
                                 .otherwise(lit(None)))

id4=id4.filter(col("id_value").isNotNull())

In [ ]:
id_f3= id_f2.union(id4)

In [ ]:
df_main = id_f3.withColumn('id_json', to_json(lit(None)))

In [ ]:
from pyspark.sql.window import Window

df_main = df_main.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("last_updated_date").desc())))

df_main = df_main.filter(F.col("rn") == 1).drop("rn")

In [ ]:
df_main.columns

In [ ]:
df_main=df_main.select(['source_system_name','source_system_customer_id','activity_id','activity_date','id_type','id_value',
'issuer','date_issued','date_reported','valid_until_date','last_updated_date','verification_flag','verification_source','active_flag','id_json'])

In [ ]:
df_main.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
master_bulk_df_records=str(df_main.select(countDistinct("source_system_customer_id")).collect()[0][0])

In [ ]:
records=df_main.count()
records

In [ ]:
output_index = "id_staging"
schema = "customermart"

import sys
try:
    
     df_main.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user",user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

In [ ]:
from pg import DB
db = DB(dbname="gpadmin", user='gpspark', passwd='spark@456', host='10.35.12.194', port= 5432)
GidCount = db.query("select count(distinct source_system_customer_id) from customermart.id_staging").getresult()

In [ ]:
table_name='id'#customermart.id_temp
source='id'
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

In [ ]:
update(conn_to,table_name,source,prod_gpdb_spark_options, schema="customermart")

In [ ]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbtable": "customermart.id",
         "server.port":"1150-1170"}


id_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.select('source_system_name','source_system_customer_id','activity_id','activity_date','id_type','id_value',
'issuer','date_issued','date_reported','valid_until_date','last_updated_date','verification_flag','verification_source','active_flag','id_json')\
.filter(col('last_updated_date').between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd'))).dropDuplicates()

In [ ]:
df_stag=df_main.select('source_system_name','source_system_customer_id','activity_id','last_updated_date','verification_flag', 'verification_source','active_flag','date_issued','date_reported','valid_until_date')

In [ ]:
id_df1=id_df.select('source_system_name','source_system_customer_id','activity_id','last_updated_date','verification_flag', 'verification_source','active_flag','date_issued','date_reported','valid_until_date')

In [ ]:
df_MissingData=df_stag.exceptAll(id_df1).withColumn('DataMisMatch',lit('Missing values at Target'))

In [ ]:
reconresult=[{
    "SrcCount":master_bulk_df_records,
    "TargetCount":GidCount[0][0],
    "TrgtMissMatchCount":df_MissingData.count()
}]

from pyspark.sql import Row
df_recon=sqlContext.createDataFrame(Row(**x) for x in reconresult)

In [ ]:
df_recon=df_recon.withColumn('Status',when((df_recon.SrcCount==df_recon.TargetCount) & (df_recon.TrgtMissMatchCount==0),'Success').otherwise('Failed'))

In [ ]:
df_recon.show()

In [ ]:
status=df_recon.select(df_recon.Status).collect()[0][0]

In [ ]:
update_progress(table_name,source,time_filter,records,start_time,starttime,status)

In [ ]:
df_main.unpersist()

In [ ]:
conn_prod.close()

In [2]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…